<a href="https://colab.research.google.com/github/Pritam-Mondal18/Ai_Comic_Crafter/blob/main/eligible_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install streamlit pyngrok


In [22]:
!ngrok authtoken 2uuTc8nX8Riw26cHpsU6EvwPE2n_56Leu6CJ2wTTC27WthpvQ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [28]:
%%writefile app.py
import streamlit as st
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

st.set_page_config(page_title="Exam Eligibility Predictor", layout="centered")

# Custom header
st.markdown("<h1 style='text-align: center; color: #4A90E2;'>🎓 Student Exam Eligibility Predictor</h1>", unsafe_allow_html=True)
st.markdown("<p style='text-align: center;'>Check student exam eligibility based on attendance percentage using AI.</p>", unsafe_allow_html=True)
st.divider()

# File upload
st.subheader("📂 Upload Attendance CSV File")
st.markdown("**Expected format:** must include a `Name` column and a `Total` column in `Attended|Total` format.")
uploaded_file = st.file_uploader("Choose a CSV file", type=["csv"])

if uploaded_file:
    df = pd.read_csv(uploaded_file)

    # Attendance threshold
    st.divider()
    st.subheader("⚙️ Attendance Criteria")
    threshold = st.slider("Select the minimum attendance % required to be eligible:", 0, 100, 75)

    # Data cleaning
    df = df.dropna(subset=["Total"])
    valid_format = df["Total"].astype(str).str.contains(r"\d+\|\d+")
    df = df[valid_format].copy()

    df[['Attended', 'Total_Classes']] = df['Total'].str.split('|', expand=True).astype(int)
    df['Attendance_Percentage'] = (df['Attended'] / df['Total_Classes']) * 100
    df['Allowed'] = (df['Attendance_Percentage'] >= threshold).astype(int)

    # Model training
    X = df[['Attendance_Percentage']]
    y = df['Allowed']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    st.success(f"✅ Model trained with accuracy: **{acc * 100:.2f}%**")

    # Confusion matrix
    st.divider()
    st.subheader("📊 Model Performance")
    cm = confusion_matrix(y_test, y_pred)
    fig, ax = plt.subplots()
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Not Allowed", "Allowed"], yticklabels=["Not Allowed", "Allowed"])
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    st.pyplot(fig)

    # Student search
    st.divider()
    st.subheader("🔍 Search Eligibility by Student Name")
    student_name = st.text_input("Enter full name (case-insensitive):")

    if student_name:
        matched = df[df['Name'].str.lower() == student_name.strip().lower()]
        if not matched.empty:
            attendance = matched.iloc[0]['Attendance_Percentage']
            prediction = model.predict([[attendance]])[0]
            status = "✅ **Allowed**" if prediction == 1 else "❌ **Not Allowed**"
            with st.expander("🎓 Student Eligibility Result", expanded=True):
                st.markdown(f"- **Student:** {student_name}")
                st.markdown(f"- **Attendance:** {attendance:.2f}%")
                st.markdown(f"- **Prediction:** {status}")
        else:
            st.warning(f"⚠️ Student '{student_name}' not found in the dataset.")

else:
    st.info("📁 Please upload a CSV file to begin.")


Overwriting app.py


In [30]:

!pkill -f ngrok

In [31]:
!ngrok authtoken 2uuTc8nX8Riw26cHpsU6EvwPE2n_56Leu6CJ2wTTC27WthpvQ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [32]:
from pyngrok import ngrok
import os
os.system("streamlit run app.py &")
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")

Streamlit app is live at: NgrokTunnel: "https://b452-35-196-186-121.ngrok-free.app" -> "http://localhost:8501"
